python setup.py sdist bdist_wheel

twine upload --skip-existing dist/*

In [1]:
import pandas as pd
import ScraperFC as sfc
import traceback

In [2]:
# scraper = sfc.WhoScored()
# try:
#     out = scraper.scrape_matches(year=2018, league='Argentina Liga Profesional')
# except:
#     traceback.print_exc()
# finally:
#     scraper.close()
# assert 'matchId' in out[list(out.keys())[-1]].keys()

In [3]:
from   IPython.display import clear_output
from   selenium import webdriver
from   selenium.webdriver.chrome.options import Options
from   selenium.webdriver.common.by import By
from   selenium.webdriver.support import expected_conditions as EC
from   selenium.webdriver.support.ui import WebDriverWait
from   webdriver_manager.chrome import ChromeDriverManager
import traceback


options = Options()
options.headless = True
options.add_argument("window-size=1400,600")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
# clear_output()


try:
    of_df, gk_df = pd.DataFrame(), pd.DataFrame()
    driver.get('https://fbref.com/en/players/')
    
    # Get the links to the alphabet sections
    print('Gathering alphabet segment links.')
    alphabet_links = set()
    for el in driver.find_elements(By.TAG_NAME, 'a'):
        href = el.get_attribute('href')
        if href and 'players' in href and len(el.text)==2:
            alphabet_links.add(href)
            
    # Visit each of the alphabet links and get the player links
    print('Gathering player reports.')
    player_links = set()
    cnt = 0
    for link in alphabet_links:
        cnt += 1
        print('{} {}/{}'.format(link.split('/')[-2], cnt, len(alphabet_links)), end='\r')
        
        if 'he' not in link.lower():
            continue
        
        driver.get(link)
        
        
        for el in driver.find_elements(By.TAG_NAME, 'p'):
            try: # skip elements that's don't have an "a" tag child
                href = el.find_element(By.TAG_NAME, 'a').get_attribute('href')
            except:
                continue
            
            if href and 'players' in href and link not in href and len(href.split('/'))>6: # Valid player link
                # player_links.add(href)
                if 'heung' not in href.lower() or 'son' not in href.lower():
                    continue
            
                driver.get(href)
                s = driver.page_source
                if 'View Complete Scouting Report' in driver.page_source: # player with scout report
                    comp_report_button = driver.find_element(
                        By.XPATH, 
                        '/html/body/div[2]/div[6]/div[2]/div[1]/div/div/div[1]/div/ul/li[2]/a'
                    )
                    driver.execute_script('arguments[0].click()', comp_report_button)
                    of_df = of_df.append(pd.Series({'url': href}), ignore_index=True)

#                 try:
#                     //*[@id="scout_summary_GK_sh"]/div/ul/li[2]/a
#                     //*[@id="scout_summary_FB_sh"]/div/ul/li[2]/a
#                     //*[@id="scout_summary_CB_sh"]/div/ul/li[2]/a
#                     //*[@id="scout_summary_MF_sh"]/div/ul/li[2]/a
#                     driver.find_element(By.XPATH, '//*[@id="scout_summary_AM_sh"]/div/ul/li[2]/a').click()
#                     //*[@id="scout_summary_FW_sh"]/div/ul/li[2]/a
#                 except:
#                     continue
#                 print(driver.current_url)
#                 break

#     # Visit each player's page and get their report
#     print('Gathering player scouting reports.')
#     for link in player_links:
#         print(link)
#         driver.get(link)
#         comp_report_button = driver.find_element(By.XPATH, '//*[@id="scout_summary_AM_sh"]/div/ul/li[2]/a')
#         print(comp_report_button)
#         break
except:
    traceback.print_exc()
finally:
    driver.close()
    print('DONE')

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Owner\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
<ipython-input-3-e78d30214fbb>:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Gathering alphabet segment links.
Gathering player reports.
DONE74/474


In [4]:
of_df.shape

(1, 1)